# Optional setup: Download the spacy model

In [17]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 6.9 MB/s eta 0:00:0000:0100:02
  Using cached spacy_curated_transformers-0.2.2-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached curated_transformers-0.1.1-py2.py3-none-any.whl.metadata (965 bytes)
  Using cached curated_tokenizers-0.0.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.m

# Source

In [18]:

from googlesearch import search
from typing import NamedTuple, Iterable

class QueryCore(NamedTuple):
    name: str
    url: str

# Example companies and products for testing
s6 = QueryCore("Singularity 6", "singularity6.com")
palia = QueryCore("Palia", "palia.com")
radai = QueryCore("Rad AI", "radai.com")
pomelo = QueryCore("Pomelo Care", "pomelocare.com")
muddyrobot = QueryCore("Muddy Robot Games", "muddyrobot.com")
rafa = QueryCore("98point6", "98poin6.com")

excluded_domains = {
    "linkedin.com",
    "twitter.com",
    "x.com",
    "youtube.com",
    "reddit.com",
    "facebook.com",
}

def news_search(core_query: QueryCore, num=10, stop=10, pause=2, last_unit='m') -> Iterable[str]:
    query = f'"{core_query.name}""'
    query += " " + " ".join(f"-site:{domain}" for domain in excluded_domains)

    yield from search(query, num=num, stop=stop, pause=pause, tbs=f'qdr:{last_unit}', extra_params={'tbm': 'nws'})

def test_news_search():
    for url in news_search(pomelo, stop=20, pause=2):
        print(url)

# test_news_search()

In [21]:
import requests
from bs4 import BeautifulSoup
from functools import lru_cache
from typing import Optional

@lru_cache(maxsize=10)
def get_article_text(url: str) -> Optional[str]:
    response = requests.get(url, timeout=5)

    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')
        article = soup.find('article')
        if article:
            return article.get_text()
    else:
        print(f"Failed to get article from {url}: {response.status_code}")
        
    return None
    
def test_get_article_text():
    print(get_article_text("https://www.pcgamesinsider.biz/news/74555/daybreak-acquires-palia-maker-singularity-6/"))


In [19]:
from pprint import pprint
from collections import Counter
import spacy

LABEL_MAPPINGS = {
    "ORG": "ORG/PRODUCT",
    "PRODUCT": "ORG/PRODUCT",
}

def munge_label(label: str) -> str:
    return LABEL_MAPPINGS.get(label, label)

# Load the trf model
nlp = spacy.load("en_core_web_trf")

def extract_entities(text: Optional[str]) -> Counter:
    if not text:
        return Counter()
    
    doc = nlp(text)

    return Counter((munge_label(ent.label_), ent.text) for ent in doc.ents)

def test_extract_entities():
    text = get_article_text("https://www.pcgamesinsider.biz/news/74555/daybreak-acquires-palia-maker-singularity-6/")
    pprint(extract_entities(text))

    text = get_article_text("https://massivelyop.com/2024/07/01/daybreak-bought-palia-studio-singularity-6-and-aims-to-bring-the-game-to-launch/")
    pprint(extract_entities(text))

# Note: The PRODUCT vs ORG on these isn't very good

/home/keith/company-detective/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f

In [22]:
# run the actual search, process each page, then aggregate both TF and DF
from collections import defaultdict
from urllib.parse import urlparse

entity_counts = Counter()
entity_document_counts = Counter()
entity_domains = defaultdict(set)

documents = {}

for url in news_search(s6, stop=10, last_unit='y'):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    text = get_article_text(url)

    documents[url] = text

    if not text:
        print(f"Failed to extract text from {url}, skipping")
        continue

    counts = extract_entities(text)

    entity_counts.update(counts)
    for entity, count in counts.items():
        entity_document_counts[entity] += 1
        entity_domains[entity].add(domain)


/home/keith/company-detective/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Failed to get article from https://store.epicgames.com/en-US/p/palia-0d428e: 403
Failed to extract text from https://store.epicgames.com/en-US/p/palia-0d428e, skipping
Failed to extract text from https://store.steampowered.com/app/2707930/Palia/, skipping
Failed to get article from https://support.palia.com/hc/en-us/articles/24877677943060--Steam-Account-Creation-Linking-and-Rewards: 403
Failed to extract text from https://support.palia.com/hc/en-us/articles/24877677943060--Steam-Account-Creation-Linking-and-Rewards, skipping
Failed to get article from https://support.palia.com/hc/en-us/articles/21548837853844--Nintendo-Switch-Account-Creation-Linking-and-Rewards: 403
Failed to extract text from https://support.palia.com/hc/en-us/articles/21548837853844--Nintendo-Switch-Account-Creation-Linking-and-Rewards, skipping
Failed to extract text from https://www.gamedeveloper.com/business/daybreak-acquires-struggling-palia-developer-singularity-6, skipping


In [23]:
top_entities = sorted(entity_document_counts.keys(), key=lambda entity: len(entity_domains[entity]), reverse=True)

for entity in top_entities[:20]:
    num_documents = entity_document_counts[entity]
    num_domains = len(entity_domains[entity])
    num_mentionds = entity_counts[entity]

    print(f"{entity}: on {num_domains} domains in {num_documents} documents ({num_mentionds} mentions total)")

('ORDINAL', 'first'): on 3 domains in 3 documents (4 mentions total)
('ORG/PRODUCT', 'Palia'): on 2 domains in 2 documents (18 mentions total)
('ORG/PRODUCT', 'Singularity 6'): on 2 domains in 2 documents (10 mentions total)
('ORG/PRODUCT', 'Singularity'): on 2 domains in 2 documents (2 mentions total)
('ORDINAL', 'second'): on 2 domains in 2 documents (2 mentions total)
('CARDINAL', 'four'): on 2 domains in 2 documents (2 mentions total)
('ORG/PRODUCT', 'Stardew Valley'): on 2 domains in 2 documents (2 mentions total)
('GPE', 'Palia'): on 2 domains in 2 documents (2 mentions total)
('CARDINAL', 'one'): on 2 domains in 2 documents (2 mentions total)
('ORG/PRODUCT', 'Final Fantasy'): on 2 domains in 2 documents (2 mentions total)
('DATE', '7/11/2024'): on 1 domains in 1 documents (1 mentions total)
('ORG/PRODUCT', 'Chapaccino Cold Brew'): on 1 domains in 1 documents (1 mentions total)
('ORG/PRODUCT', 'News'): on 1 domains in 1 documents (1 mentions total)
('PERSON', 'Cass Marshall'): on

In [24]:
documents

{'https://palia.com/news/patch-181': '   7/11/2024\xa0|\xa0\n Community \nCozy up with a Cup of Chapaccino Cold Brew\n Read More\n',
 'https://store.epicgames.com/en-US/p/palia-0d428e': None,
 'https://store.steampowered.com/app/2707930/Palia/': None,
 'https://www.polygon.com/2024/7/17/24199912/palia-favorite-cozy-game-mmo': '\n\n\n\n\nFiled under:\n\n\n\nNews\n\n\n\n\n\nEarly access life sim Palia has become my Stardew-style game of choice\n\nThe cozy game finally feels more like a MMO\n\n\n    \n      By\n      \nCass Marshall\n\n\n\n  Jul 17, 2024,  1:00pm EDT\n\n\n\n\n\n\n\n      / \nnew\n\n\n\n\n\n\n\n\n\n\n\nShare this story\n\n\n\n\nShare this on Facebook\n\n\n\n\n\n\n\n\n\n\nShare this on Reddit\n\n\n\n\n\n\n\n\nShare\nAll sharing options\n\n\n\n\n\n\nShare\nAll sharing options for:\nEarly access life sim Palia has become my Stardew-style game of choice\n\n\n\n\n\n\n\n\n\n\n\n\nReddit\n\n\n\n\n\n\n\nPocket\n\n\n\n\n\n\n\n\n\nFlipboard\n\n\n\n\n\nEmail\n\n\n\n\n\n\n\n\n\n\n\n\n

In [25]:
text = documents["https://www.polygon.com/2024/7/17/24199912/palia-favorite-cozy-game-mmo"]
print(text.strip())

Filed under:



News





Early access life sim Palia has become my Stardew-style game of choice

The cozy game finally feels more like a MMO


    
      By
      
Cass Marshall



  Jul 17, 2024,  1:00pm EDT







      / 
new











Share this story




Share this on Facebook










Share this on Reddit








Share
All sharing options






Share
All sharing options for:
Early access life sim Palia has become my Stardew-style game of choice












Reddit







Pocket









Flipboard





Email




















Image: Singularity 6






Cass Marshall
        is a news writer focusing on gaming and culture coverage, taking a particular interest in the human stories of the wild world of online games.
      




I’ve been spending a good amount of my free time lately in early-access life sim Palia. Since its early access launch, I’ve found myself returning to my human’s comfortable home to garden, hunt, and hang out with friends. Palia has become my Stardew Valley

# Information extraction with LangChain

In [26]:
from dotenv import load_dotenv
load_dotenv()


True

In [27]:
# Define the schema for extraction
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field
from enum import Enum

class HairColor(Enum):
    BROWN = 'BROWN'
    BLONDE = 'BLONDE'
    BLACK = 'BLACK'
    GREY = 'GREY'
    WHITE = 'WHITE'
    RED = 'RED'
    OTHER = 'OTHER'

    def __str__(self):
        return self.value

class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person")

    hair_color: Optional[HairColor] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[float] = Field(
        default=None, description="Height measured in meters"
    )



In [28]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)



In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Person)

In [11]:
text = "Alan Smith is 6 feet tall and has blond hair."
runnable.invoke({"text": text})

Person(name='Alan Smith', hair_color=<HairColor.BLONDE: 'BLONDE'>, height_in_meters=1.83)

# Information extraction for the actual articles

In [46]:
# Define the schema for extraction
from typing import Optional, List

from langchain_core.pydantic_v1 import BaseModel, Field
from enum import Enum
from datetime import date

class Sentiment(Enum):
    POSITIVE = "POSITIVE"
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"

    def __str__(self):
        return self.value

class CustomDate(BaseModel):
    """A structued date"""
    year: int = Field(default=None, description="The year")
    month: int = Field(default=None, description="The month starting from 1")
    day: int = Field(default=None, description="The day of the month starting from 1")

    @classmethod
    def from_date(cls, d: date) -> 'CustomDate':
        return cls(year=d.year, month=d.month + 1, day=d.day + 1)

    def to_date(self) -> date:
        return date(year=self.year, month=self.month - 1, day=self.day - 1)

class OpinionArticle(BaseModel):
    """An opinion piece such as a product review, game review, or speculation about a business."""

    author_name: Optional[str] = Field(default=None, description="The name of the author")
    publication_date: Optional[CustomDate] = Field(default=None, description="The publication date")
    title: Optional[str] = Field(default=None, description="The title of the article")
    sentiment: Sentiment = Field(default=None, description="The sentiment of the article")

class Acquisition(BaseModel):
    """One company acquires another company."""

    acquiring_company: Optional[str] = Field(default=None, description="The name of the acquiring company")
    acquired_company: Optional[str] = Field(default=None, description="The name of the acquired company")
    acquisition_date: Optional[CustomDate] = Field(default=None, description="The date of the acquisition")

    # TODO: Structured
    acquisition_amount: Optional[str] = Field(default=None, description="The amount of the acquisition")

class Partnership(BaseModel):
    """A business partnership."""

    company_1: Optional[str] = Field(default=None, description="The name of the first company in the partnership")
    company_2: Optional[str] = Field(default=None, description="The name of the second company in the partnership")
    partnership_date: Optional[CustomDate] = Field(default=None, description="The date of the partnership")

    summary: Optional[str] = Field(default=None, description="A one-sentence extracted summary of the partnership")

class Fundraising(BaseModel):
    """A business that raises investment."""

    company: Optional[str] = Field(default=None, description="The name of the company that raised funds")
    fundraising_date: Optional[CustomDate] = Field(default=None, description="The date of the fundraising event")

    # TODO: Structured
    fundraising_amount: Optional[str] = Field(default=None, description="The amount of funds raised")

class Quote(BaseModel):
    """A person being quoted."""

    speaker: Optional[str] = Field(default=None, description="The person who made the quote")
    text: Optional[str] = Field(default=None, description="The text of the quote")

class BusinessNewsArticle(BaseModel):
    """News about a business."""

    author_name: Optional[str] = Field(default=None, description="The name of the author")
    publication_date: Optional[CustomDate] = Field(default=None, description="The publication date")
    title: Optional[str] = Field(default=None, description="The title of the article")
    quotes: List[Quote] = Field(default=[], description="Quotes included in the article (possibly empty)")

    acquisition_event: Optional[Acquisition] = Field(default=None, description="The acquisition event, if applicable")
    partnership_event: Optional[Partnership] = Field(default=None, description="The partnership event, if applicable")
    fundraising_event: Optional[Fundraising] = Field(default=None, description="The fundraising event, if applicable")



In [48]:
# Classification of the page
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You're an expert algorithm to classify passages about a particular company or product.
            Only extract the properties mentioned in the 'Classification' function.
            """,
        ),
        (
            "human", 
            """
            Company: {company_name}
            Passage: 
            {text}
            """
            ),
    ]
)

class ArticleCategory(Enum):
    PRODUCT_UPDATE = "PRODUCT_UPDATE"
    BUSINESS_NEWS = "BUSINESS_NEWS"
    OPINION = "OPINION"
    OTHER = "OTHER"
    UNRELATED_TO_COMPANY = "UNRELATED_TO_COMPANY"
    
class Classification(BaseModel):
    sentiment: Sentiment = Field(description="The sentiment of the text")
    language: str = Field(description="The language the text is written in")
    category: ArticleCategory = Field(description="The category of the article with respect to the company")



In [49]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Classification)

In [50]:
runnable.invoke({"text": get_article_text("https://massivelyop.com/2024/07/01/daybreak-bought-palia-studio-singularity-6-and-aims-to-bring-the-game-to-launch/"), "company_name": "Singularity 6"})

Classification(sentiment=<Sentiment.NEUTRAL: 'NEUTRAL'>, language='English', category=<ArticleCategory.PRODUCT_UPDATE: 'PRODUCT_UPDATE'>)